# Лабораторная работа №2. Шут Артём

In [1]:
import numpy as np
from numpy import linalg as la
import copy
import math
from matplotlib import pyplot as plt
from prettytable import PrettyTable as pt
from math import sqrt

#### Система согласно варианту:
\begin{cases}   
    3.241x_1 + 0.197x_2 + 0.643x_3 + 0.236x_4 = 0.454; \\
    0.257x_1 + 3.853x_2 + 0.342x_3 + 0.427x_4 = 0.371;\\
    0.324x_1 + 0.317x_2 + 2.793x_3 + 0.238x_4 = 0.465; \\
    0.438x_1 + 0.326x_2 + 0.483x_3 + 4.229x_4 = 0.822.
\end{cases}

#### Матрица системы:

In [2]:
A = np.array([[3.241, 0.197, 0.643, 0.236], 
              [0.257, 3.853, 0.342, 0.427], 
              [0.324, 0.317, 2.793, 0.238],
              [0.438, 0.326, 0.483, 4.229]])
b = np.array([0.454, 0.371, 0.465, 0.822])

In [3]:
print(la.solve(A, b))

[0.09783369 0.0596282  0.13437065 0.16429629]


# Задание 1

### Метод Гаусса

In [4]:
def gauss(a, B):
    A = copy.deepcopy(a)
    b = copy.deepcopy(B)
    
    n = len(A)

    for i in range(0, n):
        for k in range(i + 1, n):
            c = -A[k][i] / A[i][i]
            for j in range(i, n):
                if i == j:
                    A[k][j] = 0
                    
                else:
                    A[k][j] += c * A[i][j]
                
            b[k] += c * b[i]
            
    x = [0 for i in range(n)]
    for i in range(n - 1, -1, -1):
        x[i] = b[i] / A[i][i]
        for k in range(i - 1, -1, -1):
            b[k] -= A[k][i] * x[i]       
            
    return x

### Метод Гаусса-Йордана

In [5]:
def gauss_jordan(a, B):
    A = copy.deepcopy(a)
    b = copy.deepcopy(B)

    n = len(A)

    E = np.array([[0.0 for i in range(n)] for i in range(n)])
    for i in range(n):
        E[i][i] = 1.0
    A = np.concatenate((A, E), 1)

    for i in range(n):
        for k in range(i + 1, n):
            c = -A[k][i] / A[i][i]
            for j in range(i, 2 * n):
                if i == j:
                    A[k][j] = 0
                else:
                    A[k][j] += c * A[i][j]

    for i in range(n):
        c = A[i][i]
        for j in range(2 * n):
            A[i][j] /= c
            
    for i in range(n - 1, 0, -1):
        for k in range(i - 1, -1, -1):
            c = -A[k][i]
            for j in range(i, 2 * n):
                if i == j:
                    A[k][j] = 0
                else:
                    A[k][j] += c * A[i][j]                     

    A = np.array([[A[i][j] for j in range(n, 2 * n)] for i in range(n)])
    return A.dot(b)

### Метод Якоби

In [6]:
def jacobi(A, b, x, eps, n_iter):
    D = np.diag(np.diag(A))
    LU = A - D
    for n in range(n_iter):
        D_inv = np.diag(1 / np.diag(D))
        x_prev = x
        x = np.dot(D_inv, b - np.dot(LU, x))
        if np.linalg.norm(x - x_prev) < eps:
            break  
    return (x, n + 1, la.norm(x - x_prev))

### Метод Гаусса-Зейделя

In [7]:
def gauss_seidel(A, b, x, eps, n_iter):
    N = len(A)
    for n in range(n_iter):
        x_prev = copy.deepcopy(x)
        for i in range(len(A)):
            sn1 = sum(A[i][j] * x[j] for j in range(i))
            sn = sum(A[i][j] * x_prev[j] for j in range(i + 1, N))
            x[i] = (b[i] - sn1 - sn) / A[i][i]
        if la.norm(x - x_prev) <= eps:
            break
    return (x, n + 1, la.norm(x - x_prev))

### Сравнение

In [8]:
print("We will use standart Python func to solve SLE to the estimate the discrepancy")

print("\nGauss method:")
print("Result:", gauss(A, b))
print("The discrepancy:", la.norm(gauss(A, b) - la.solve(A, b)))

print("\nGauss-Jordan method:")
print("Result:", gauss(A, b))
print("The discrepancy:", la.norm(gauss_jordan(A, b) - la.solve(A, b)))

print("\nJacobi method:")
X, N, Eps = jacobi(A,b,np.zeros(len(A)),1e-16,100)
print("The result:", X, "\nThe number of iterations:", N,
     "\nThe discrepancy:", Eps)

print("\nGauss-Seidel method:")
X, N, Eps = gauss_seidel(A,b,np.zeros(len(A)),1e-16,100)
print("The result:", X, "\nThe number of iterations:", N,
     "\nThe discrepancy:", Eps)

We will use standart Python func to solve SLE to the estimate the discrepancy

Gauss method:
Result: [0.0978336904561916, 0.0596282018593483, 0.13437065034901355, 0.16429629360498157]
The discrepancy: 5.721958498152797e-17

Gauss-Jordan method:
Result: [0.0978336904561916, 0.0596282018593483, 0.13437065034901355, 0.16429629360498157]
The discrepancy: 6.245004513516506e-17

Jacobi method:
The result: [0.09783369 0.0596282  0.13437065 0.16429629] 
The number of iterations: 32 
The discrepancy: 5.238750013840908e-17

Gauss-Seidel method:
The result: [0.09783369 0.0596282  0.13437065 0.16429629] 
The number of iterations: 14 
The discrepancy: 1.3877787807814457e-17


# Задание 2

### Метод сопряжённых градиентов

In [9]:
def CG(A, b, x, eps = 1e-9, n_iter = 100):
    d = r = b - A.dot(x)
    norm_b = la.norm(b)
    for n in range(n_iter):
        alpha = r.dot(r)/d.dot(A.dot(d))
        x = x + alpha * d
        if la.norm(r)/norm_b < eps:
            return x, n + 1, eps
        r_last = r
        r = r - alpha * A.dot(d)
        d = r + r.dot(r)/r_last.dot(r_last)*d
    print("Limit iterations")
    return x, n_iter, la.norm(r)/norm_b

### Метод сопряжённых градиентов с переобуславливанием

In [10]:
def PCG(A, M, b, x, eps = 1e-9, n_iter = 100):
    r = b - A.dot(x)
    d = M.dot(r)
    norm_b = la.norm(b)
    for n in range(n_iter):
        alpha = r.dot(M.dot(r))/d.dot(A.dot(d))
        x = x + alpha * d
        if la.norm(r)/norm_b < eps:
            return x, n + 1, eps
        r_last = r
        r = r - alpha * A.dot(d)
        d = M.dot(r) + r.dot(M.dot(r))/r_last.dot(M.dot(r_last))*d
    print("Limit iterations")
    return x, n_iter, la.norm(r)/norm_b

### Сравнение

In [11]:
print("Conjugate Gradient method:")
print("The condition number of matrix A:", la.cond(A))
X, N, Eps = CG(A, b, np.zeros(len(b)))
print("The result of :", X, "\nThe number of iterations:", N,
     "\nThe discrepancy:", Eps)

M = np.zeros((4,4))
for i in range(len(A)):
    M[i][i] = A[i][i]
M = la.inv(M)

print("\nPreconditioned Conjugate Gradient method (1):")
print("The condition number:", la.cond(M.dot(A)))
X, N, Eps = PCG(A, M, b, np.zeros(len(b)))
print("The result:", X, "\nThe number of iterations:", N,
     "\nThe discrepancy:", Eps)

M = np.zeros((4,4))
for i in range(len(A)):
    M[i][i] = 1/A[i][i]
M = la.inv(M)

print("\nPreconditioned Conjugate Gradient method (2):")
print("The condition number:", la.cond(M.dot(A)))
X, N, Eps = PCG(A, M, b, np.zeros(len(b)))
print("The result:", X, "\nThe number of iterations:", N,
     "\nThe discrepancy:", Eps)

Conjugate Gradient method:
The condition number of matrix A: 1.9383507261324149
The result of : [0.09783369 0.0596282  0.13437065 0.16429629] 
The number of iterations: 21 
The discrepancy: 1e-09

Preconditioned Conjugate Gradient method (1):
The condition number: 1.5586818493229424
The result: [0.09783369 0.0596282  0.13437065 0.16429629] 
The number of iterations: 13 
The discrepancy: 1e-09

Preconditioned Conjugate Gradient method (2):
The condition number: 2.7097301155073943
The result: [0.09783369 0.0596282  0.13437065 0.16429629] 
The number of iterations: 33 
The discrepancy: 1e-09


### Вывод

Если число обусловленности велико, то переобуславливание целесобразно, - если мало, то нет. Как мы видим, при переобуславливании мы можем как уменьшить, так и увеличить число обусловленности матрицы, и это сказывается на точности метода, на количестве итераций до получения результата с заданной точностью. Более того, при больших числах обусловленности метод может и не сходиться в принципе при некоторых входных данных.

# Задание 3

In [13]:
n = 3
H = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        H[i][j] = 1/(1 + i + j)
        
print(la.cond(H))
print(la.norm(H - gauss_jordan(gauss_jordan(H, np.eye(len(H))), np.eye(len(H)))))

524.0567775860644
3.605234528942684e-15
